In [ ]:
!pip install ultralytics
!pip install roboflow


In [ ]:
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from roboflow import Roboflow
from ultralytics import YOLO

In [ ]:
model = YOLO('yolo11x.pt')
result = model.track('/content/Tennis.mp4', conf = 0.25, save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/231) /content/Tennis.mp4: 384x640 7 persons, 2 sports balls, 1 couch, 1 clock, 73.4ms
video 1/1 (frame 2/231) /content/Tennis.mp4: 384x640 7 persons, 1 sports ball, 1 couch, 1 clock, 19.3ms
video 1/1 (frame 3/231) /content/Tennis.mp4: 384x640 6 persons, 1 sports ball, 1 couch, 1 clock, 19.6ms
video 1/1 (frame 4/231) /content/Tennis.mp4: 384x640 6 persons, 1 sports ball, 1 couch, 1 clock, 19.5ms
video 1/1 (frame 5/231) /content/Tennis

In [ ]:
!pip install gdown
!gdown "https://drive.google.com/uc?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu"


Downloading...
From (original): https://drive.google.com/uc?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu
From (redirected): https://drive.google.com/uc?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&confirm=t&uuid=b5d38f91-b575-462c-9a31-e1d87833cc0c
To: /content/tennis_court_det_dataset.zip
100% 7.26G/7.26G [02:28<00:00, 48.7MB/s]


In [ ]:
!unzip /content/tennis_court_det_dataset.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: data/images/JNKp7sCdQlY_2200.png  
  inflating: data/images/JNKp7sCdQlY_2250.png  
  inflating: data/images/JNKp7sCdQlY_250.png  
  inflating: data/images/JNKp7sCdQlY_300.png  
  inflating: data/images/JNKp7sCdQlY_350.png  
  inflating: data/images/JNKp7sCdQlY_400.png  
  inflating: data/images/JNKp7sCdQlY_450.png  
  inflating: data/images/JNKp7sCdQlY_50.png  
  inflating: data/images/JNKp7sCdQlY_500.png  
  inflating: data/images/JNKp7sCdQlY_550.png  
  inflating: data/images/JNKp7sCdQlY_600.png  
  inflating: data/images/JNKp7sCdQlY_650.png  
  inflating: data/images/JNKp7sCdQlY_700.png  
  inflating: data/images/JNKp7sCdQlY_750.png  
  inflating: data/images/juXbdW7z0WA_100.png  
  inflating: data/images/juXbdW7z0WA_1050.png  
  inflating: data/images/juXbdW7z0WA_1100.png  
  inflating: data/images/juXbdW7z0WA_200.png  
  inflating: data/images/juXbdW7z0WA_350.png  
  inflating: data/images/juXbdW7z0WA

### Start training

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import cv2
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Device:", device)

Device: cuda


In [ ]:
class KeypointsDataset(Dataset):
  def __init__(self, img_dir, data_file):
    self.img_dir = img_dir
    with open(data_file, "r") as f:
      self.data = json.load(f)

    self.transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):

      item = self.data[idx]
      img_path = f"{self.img_dir}/{item['id']}.png"
      img = cv2.imread(img_path)
      if img is None:
          raise FileNotFoundError(f"Image not found at path: {img_path}")

      h, w = img.shape[:2]

      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

      img = self.transforms(img)

      kps = np.array(item['kps']).flatten()
      kps = kps.astype(np.float32)

      kps[::2] *= 224.0 / w
      kps[1::2] *= 224.0 / h

      return img, kps



In [ ]:
train_dataset = KeypointsDataset("/content/data/images", "/content/data/data_train.json")
val_dataset = KeypointsDataset("/content/data/images", "/content/data/data_val.json")

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = 8, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 8, shuffle = True)

## Create Model

In [ ]:
model = models.resnet50(pretrained = True)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2)
model = model.to(device)

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
epochs=25
for epoch in range(epochs):
  for i, (imgs,kps) in enumerate(train_loader):
    imgs = imgs.to(device)
    kps = kps.to(device)
    optimizer.zero_grad()
    outputs = model(imgs)
    loss = criterion(outputs, kps)
    loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14451.4853515625
Epoch 0, iter 10, loss: 14538.482421875
Epoch 0, iter 20, loss: 14171.12109375
Epoch 0, iter 30, loss: 13510.0087890625
Epoch 0, iter 40, loss: 14539.396484375
Epoch 0, iter 50, loss: 12992.974609375
Epoch 0, iter 60, loss: 13139.8310546875
Epoch 0, iter 70, loss: 11997.6943359375
Epoch 0, iter 80, loss: 12441.0166015625
Epoch 0, iter 90, loss: 11768.150390625
Epoch 0, iter 100, loss: 10910.9365234375
Epoch 0, iter 110, loss: 10791.13671875
Epoch 0, iter 120, loss: 10148.9013671875
Epoch 0, iter 130, loss: 9818.404296875
Epoch 0, iter 140, loss: 9599.296875
Epoch 0, iter 150, loss: 9906.80859375
Epoch 0, iter 160, loss: 8510.0703125
Epoch 0, iter 170, loss: 8626.04296875
Epoch 0, iter 180, loss: 9149.2880859375
Epoch 0, iter 190, loss: 8575.6015625
Epoch 0, iter 200, loss: 7802.6435546875
Epoch 0, iter 210, loss: 7315.232421875
Epoch 0, iter 220, loss: 7290.23583984375
Epoch 0, iter 230, loss: 7675.02392578125
Epoch 0, iter 240, loss: 6735.519042

In [ ]:
torch.save(model.state_dict(), "keypoints_model.pth")

In [ ]:
import torch
import cv2
import numpy as np
from torchvision import transforms, models

# Define the model architecture (must match the training setup)
class KeypointModel(torch.nn.Module):
    def __init__(self):
        super(KeypointModel, self).__init__()
        self.backbone = models.resnet50(pretrained=True)
        # Replace the final fully connected layer with one for keypoints (14 x 2)
        self.backbone.fc = torch.nn.Linear(self.backbone.fc.in_features, 14 * 2)

    def forward(self, x):
        return self.backbone(x)

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = KeypointModel().to(device)

# Load the state_dict for the backbone
pretrained_dict = torch.load('/content/keypoints_model.pth', map_location=device)
model_dict = model.state_dict()

# Filter out unnecessary keys
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}

# Update the model's state_dict
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)

model.eval()

# Define preprocessing transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  # Match training input size (224x224)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Open the video file
cap = cv2.VideoCapture('/content/Tennis.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/content/Tennis_predictions.mp4', fourcc, fps, (width, height))

# Process video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    input_frame = transform(frame).unsqueeze(0).to(device)

    # Predict keypoints
    with torch.no_grad():
        keypoints = model(input_frame).cpu().numpy().reshape(-1, 2)  # Reshape to (14, 2)

    # Debugging: Print the original keypoints (normalized)
    print(f"Original keypoints (normalized): {keypoints}")

    # Postprocess: Rescale keypoints and draw them on the frame
    for (x, y) in keypoints:
        # Rescale keypoints to the original resolution (multiply by width and height)
        x_rescaled = int(x * width)  # Scale based on original width
        y_rescaled = int(y * height)  # Scale based on original height

        # Debugging: Print rescaled keypoints
        print(f"Rescaled keypoint: ({x_rescaled}, {y_rescaled})")

        # Draw keypoints on the frame
        cv2.circle(frame, (x_rescaled, y_rescaled), 5, (0, 255, 0), -1)  # Green color

    # Write the frame with keypoints
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

# Save the output video
from google.colab import files
files.download('/content/Tennis_predictions.mp4')


Выходные данные были обрезаны до нескольких последних строк (5000).
 [    0.41738     0.15329]
 [    0.33382     0.19357]]
Rescaled keypoint: (-505, -62)
Rescaled keypoint: (-33, -167)
Rescaled keypoint: (-320, 71)
Rescaled keypoint: (1974, -247)
Rescaled keypoint: (905, 389)
Rescaled keypoint: (-688, -160)
Rescaled keypoint: (-90, 73)
Rescaled keypoint: (-844, 538)
Rescaled keypoint: (1150, -515)
Rescaled keypoint: (246, -160)
Rescaled keypoint: (675, 118)
Rescaled keypoint: (-741, 923)
Rescaled keypoint: (801, 165)
Rescaled keypoint: (640, 209)
Original keypoints (normalized): [[    -0.2576   -0.038915]
 [  -0.022308    -0.15771]
 [    -0.1344   0.0037989]
 [      1.077    -0.18136]
 [    0.43435     0.36867]
 [   -0.43479   -0.079257]
 [  -0.068494     0.08991]
 [   -0.44293     0.39416]
 [    0.67065    -0.46909]
 [    0.10973    -0.13627]
 [    0.41213    0.083469]
 [   -0.38447     0.87288]
 [    0.34846     0.16029]
 [    0.34208     0.21226]]
Rescaled keypoint: (-494, -42)
Resc

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>